Please visualize the notebook in nbviewer through this link : 

In [2]:
#libraries 
import pandas as pd
import gzip
from tqdm import tqdm

from datetime import datetime
from datetime import date, time
from dateutil.parser import parse
import numpy as np
import pandas as pd


from bokeh.plotting import figure, output_notebook, show,output_file,gridplot
from bokeh.models import DatetimeTickFormatter,HoverTool,Span

import json
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

## 1-Metadata:
Before taking a closer look at the ratings we will consider the metadata dataset. In this dataset we will amongst other things find information about title, price, description, and the products each product is related to. In order to examine the metadata, we load part of it into a data frame to be able to see it and do some descriptive statistics. 

In [4]:

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta = getDF('meta_Electronics.json.gz')
meta.head(5)

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


We see that we have nine columns in total:

- asin (This is the id of the product)
- brand
- categories (list of different categories, that the product belongs to)
- description (can be very long, e.g. a book summary)
- imUrl (URL to picture of the product)
- price
- related (dictionary of different features concerning related products. In particular: also_bpught, also_viewed, boght_together)
- salesRank (ranking of product within their category based on how much it sells)
- title

Most of these things are observable from the dataset, but some things needed a seperate explaination such as the salesRank, which we found on Amazon.com.

We are interested only in a subset of variables, namely the code of the products (asin) , their titles, their sub-categories, their sales ranks, their prices.

In [5]:
#extract the sub-categories
meta['sub_category_1']=list(map(lambda x:x[0][1],meta.categories))
meta['sub_category_0']=list(map(lambda x:x[0][0],meta.categories))


In [12]:
#extract the sales category (just in case it is no the same as the main category)and the sales rank 
sales_cat=[]
sales_rank=[]
for i in range(len(meta)):
    if (type(meta.salesRank[i])==dict) :
        if(len(meta.salesRank[i])>0):
            sales_cat.append(list(meta.salesRank[i].keys())[0])
            sales_rank.append(list(meta.salesRank[i].values())[0])
        else:
            sales_cat.append('unkown')
            sales_rank.append(None)
    else:
        sales_cat.append('unkown')
        sales_rank.append(None)

In [13]:
meta['sales_category']=sales_cat
meta['sales_rank']=sales_rank

In [14]:
meta.head(1)

,asin,imUrl,description,categories,title,price,salesRank,related,brand,sub_category_1,sub_category_0,sales_category,sales_rank
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN,Computers & Accessories,Electronics,unkown,NaN


In [21]:
#extract only the metadata we need 
meta_use =meta [['asin','title','price','sub_category_1','sales_rank']]

In [22]:
meta_use.head(10)

,asin,title,price,sub_category_1,sales_rank
0,0132793040,Kelby Training DVD: Mastering Blend Modes in A...,NaN,Computers & Accessories,NaN
1,0321732944,Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,Computers & Accessories,NaN
2,0439886341,Digital Organizer and Messenger,8.15,Computers & Accessories,144944.0
3,0511189877,CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,Accessories & Supplies,NaN
4,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,299.99,GPS & Navigation,NaN
5,0558835155,Polaroid Pbm2200 PC / Gaming Stereo Headphones...,13.95,Accessories & Supplies,NaN
6,0594012015,Barnes &amp; Noble NOOK Power Kit in Carbon BN...,32.10,eBook Readers & Accessories,NaN
7,0594017580,Jonathan Adler Punctuation Cover in Black,78.83,eBook Readers & Accessories,NaN
8,0594450209,Barnes &amp; Noble 7&quot; Lautner Chocolate N...,15.55,eBook Readers & Accessories,NaN
9,0594287995,Kate Spade Rain or Shine Magazine Cover for No...,NaN,eBook Readers & Accessories,NaN


### Remarks 
As we can see the data presents many flaws: 
- some of the prices are missing. 
- the sales rank and category arer not always available.However, we assume that the sales category is always Electronics for this dataset.
- some of the data is put in another category than Electronics. We have a closer look at these products and realize that they do fit in the Electronics category anyway.

In [23]:
#meta[meta.sub_category_0!='Electronics'].head()

In [11]:
#any(meta.sub_category_0!=meta.sales_category)

True

## 2-Reviews dataset
### Download the data :

We now move the review dataset.
The review data is divided in several json files according to the Amazon category of the corresponding products (books, videos, … ). For now, we are going to focus on the Amazon_instant_video category - which is small enough to run on our computers- before extending our analysis to the others. WE note that we use the 5-core dataset which means that teh original dataset has been reduced so that each of the remaining users and items have 5 reviews each.

We start by downloading the file. Then we store it into a pandas dataframe which will enable us to have a clear overview of the data.


In [25]:
df = getDF('reviews_Electronics_5.json.gz')
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


Both the reviewr and the product are identified by codes, respectively `reviewerID`and `asin`.We also have the actual name of the reviewer.
The text of the reviews as well as the corresponding summaries are represented by strings.
Each review is characterized by: 
- the name and the id of the reviewer
- the identification of the product
- a text and a summary 
- an overall rating score
- a list of two items giving thethe number of people that found the review usefuland the total number of people who read it.
- the review time , given both in unix format and as a string.

Our first step is to change the type of review time to datetime format.


In [ ]:
#the dates are initially strings --> convert them to datetime format 
df.reviewTime=pd.to_datetime(df.reviewTime)

In [ ]:
#which are the variables that contain Nan values : 
print(df.shape)
df.isnull().any()

We notice that some of the reviewer names are missing but since the reviewer Id are available anyway, this is not an issue.

### enrich with new variables

We will now enrich the dataframe with new variables, that we give us a more meaningful insight into the reviews

In [ ]:

df['year']=list(map(lambda x:x.year,df.reviewTime))
df['length_review']=list(map(lambda x:len(x.split(' ')),df.reviewText))


Instead of keeping a list $[a,b]$ giving the number of people who found the review helpful $(a)$ and total number of people who rated this review as helpful or not $(b)$, we compute a helpfulness score as $\frac{a}{b}$.
if nobody rated the helpfulness of the review (i.e when $b=0$) we set the score to None.

In [ ]:
l=[]
for i in df.index:
    if df.helpful[i][1]==0:
        l.append(None)
    else:
        l.append(df.helpful[i][0]/df.helpful[i][1])
df['helpfulness']=l
del df['helpful']
del df['unixReviewTime']

### sentiment analysis by review 

In [ ]:
df.head(5)

## 3- Merge the two dataframes
The final Step consists in merging the metadata and reviews dataframe.


In [ ]:
#merge 
df_com= pd.merge(df,meta_use,how='left',on='asin')
df_com.head()


Now that the data has been processed, we store the dataframe in a pickle file, so that it is ready to use for the rest of the project.

In [24]:
#picke the merged dataframe
pickling_on = open("Electronics_merged.pickle","wb")
pickle.dump(df_com, pickling_on)
pickling_on.close()

# Time visualization:

In this section we will define the functions necessary to plot Amazon reviews timeline.

In [27]:
#load the dataframe 
pickle_off = open("electronics_meta_sentiment.pickle","rb")
df = pickle.load(pickle_off)

The idea is to compute the daily median value of a given variable for each sub-category and plot it from 1996-2004 to see if there is any noticeable trend.

In [33]:
evolution= df.groupby(['sub_category_1','reviewTime']).median()
evolution['number_reviews']=df.groupby(['sub_category_1','reviewTime']).count().asin
evolution.year=list(map(lambda x:int(x) , evolution.year))


After ploting the timeline for all the sub-categories, we notice that the size of some of them is too small to give readable plots. We will then restrict ourselves to the 4 largest categories, which we select below.

In [34]:

categories=list(set(df.sub_category_1))
categories=['Computers & Accessories' , 'Accessories & Supplies' , 'Camera & Photo' , 'Portable Audio & Video']
colors=['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']


In [35]:
def compare_timeline(p,variable,category,c='navy'):
    data=evolution.loc[category,variable]
    avg=data.rolling(15,center=True).median()
    hover=HoverTool(tooltips=[( 'Year',   '@year'),( category,  '@'+str(variable) ),],)
    p.line(data.index.values,avg,color=c,legend=str(category))
    p.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d',years='%Y',months='%B')
    return p

In [36]:
p = figure(width=1000, height=500, x_axis_type="datetime",tools=[hover,'pan','wheel_zoom','box_zoom','reset'])
output_notebook()
i=0
for cat in categories:
    p=compare_timeline(p,'number_reviews',str(cat),c=colors[i])
    i=i+1
p.title.text='evolution of the median daily number of reviews' 
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
show(p)

Loading BokehJS ...

In [39]:
hover=''
p = figure(width=1000, height=500, x_axis_type="datetime",tools=[hover,'pan','wheel_zoom','box_zoom','reset'])
output_notebook()
i=0
for cat in categories:
    p=compare_timeline(p,'length_review',str(cat),c=colors[i])
    i=i+1
p.title.text='evolution of the median daily length of reviews' 
p.legend.location = "top_left"
p.grid.grid_line_alpha=0
p.xaxis.axis_label = 'Date'
show(p)

Loading BokehJS ...

## Seasonal Decomposition


In [41]:
from statsmodels.tsa.seasonal import seasonal_decompose

/Users/sonia/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
df1=df[(df.year>2007)&(df.year<2012)]
ev1= df1.groupby(['reviewTime']).median()
ev1['number_reviews']=df1.groupby(['reviewTime']).count().asin
ev1.year=list(map(lambda x:int(x) , ev1.year))
variable='number_reviews'
data1=pd.Series(ev1[variable])
res1 = seasonal_decompose(data1,freq=365)



In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(4,1,1)
plt.plot(res1.observed)
plt.ylabel('initial')
plt.subplot(4,1,2)
plt.plot(res1.trend)
plt.ylabel('Trend')
plt.subplot(4,1,3)
plt.plot(res1.seasonal)
plt.ylabel('Seasonal')
plt.subplot(4,1,4)
plt.plot(res1.resid)
plt.ylabel('Residual')

In [ ]:
res2=seasonal_decompose(res1.resid.dropna(),freq=7)
plt.figure(figsize=(20,10))
plt.subplot(4,1,1)
plt.plot(res3.observed)
plt.ylabel('initial')
plt.subplot(4,1,2)
plt.plot(res3.trend)
plt.ylabel('Trend')
plt.subplot(4,1,3)
plt.plot(res3.seasonal)
plt.ylabel('Seasonal')
plt.subplot(4,1,4)
plt.plot(res3.resid)
plt.ylabel('Residual')

In [ ]:
res3.seasonal[0:30].plot()

# Correlations

In [ ]:
score=df[['sub_category_1','overall','length_review','price','sentiment_com','sentiment_pos','sentiment_neg','helpfulness']].groupby('sub_category_1').agg(['mean','std'])
score=score.iloc[[1,2,7,14]]


In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
score['sentiment_pos']['mean'].plot.bar(yerr=score['sentiment_pos']['std'])
plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('Positive sentiment of the review')
plt.subplot(2,2,2)
score['sentiment_neg']['mean'].plot.bar(yerr=score['sentiment_neg']['std'])
plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('Negative sentiment of the review')
plt.subplot(2,2,3)
score['helpfulness']['mean'].plot.bar(yerr=score['helpfulness']['std'])
plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('helpfulness of the review')
plt.subplot(2,2,4)
score['length_review']['mean'].plot.bar(yerr=score['length_review']['std'])

plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('length of the review')

In [ ]:
sc_overall=df.groupby('overall').agg(['mean','std'])
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
sc_overall['sentiment_pos']['mean'].plot.bar(yerr=sc_overall['sentiment_pos']['std'])
plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('Positive sentiment of the review')
plt.subplot(2,2,2)
sc_overall['sentiment_neg']['mean'].plot.bar(yerr=sc_overall['sentiment_neg']['std'])
plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('Negative sentiment of the review')
plt.subplot(2,2,3)
sc_overall['helpfulness']['mean'].plot.bar(yerr=sc_overall['helpfulness']['std'])
plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('helpfulness of the review')
plt.subplot(2,2,4)
sc_overall['length_review']['mean'].plot.bar(yerr=sc_overall['length_review']['std'])

plt.axhline(y=0, xmin=0, xmax=1, hold=None)
plt.title('length of the review')